In [1]:
# Donchian Weekly Classic Trend Following System
import gta_indicator
import gta_price
import gta_signal
import gta_size
import numpy as np
import pandas as pd

In [2]:
# Trade parameters.
exchange = 'LSE'
tidm = 'HSV'
timeframe = 'Weekly'
period = 48 # Look back period.
position_size = 7500  # Position size in major currency unit.
risk_pct = 0.2 # Percentage risk per trade.
commission = 11.95  # Commission per trade.
sduty = 0.5  # Stamp Duty percentage.

In [3]:
# Import weekly closing prices.
price = gta_price.weekly(exchange, tidm)

In [4]:
# Donchian channels.
dc = gta_indicator.donchian(price, period)

In [5]:
# Buy & sell signals.
ds = gta_signal.state(price, dc)
ds.loc['2001-11':'2002-01']

,buy,sell,state
date,,,
2001-11-02,0,0,0
2001-11-09,0,0,0
2001-11-16,0,0,0
2001-11-23,0,0,0
2001-11-30,0,0,0
2001-12-07,1,0,0
2001-12-14,1,0,0
2001-12-21,0,0,0
2001-12-28,0,0,0
